In [1]:
import numpy as np

# Q 1(.5 point) Define a numpy array a such that np.dot(a, b) =
array([

   [2, 5, 8, 9],

   [2, 5, 8, 9],

   [2, 5, 8, 9],

   [2, 5, 8, 9]])

In [4]:
b= np.array([[2, 5 ,8, 9] ])
print(b)
print(b.shape)
m,n = b.shape

[[2 5 8 9]]
(1, 4)


In [5]:
a = np.ones((n,m))
print(a)

[[1.]
 [1.]
 [1.]
 [1.]]


In [6]:
np.dot(a,b)

array([[2., 5., 8., 9.],
       [2., 5., 8., 9.],
       [2., 5., 8., 9.],
       [2., 5., 8., 9.]])

# In this homework
- we'll collect climate change related tweet from twitter using REST API
- Use tweets to populate Pandas DataFrames
- Perform some elementary pandas operation to check data sanity


**Please read**

- https://developer.twitter.com/en/developer-terms/more-on-restricted-use-cases


Before proceeding with homework.

<font size="6" color='red'> Create new app at https://apps.twitter.com/app/new  (You may have to create twitter account before this) </font>





[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)

# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


Click on app details and keys and token

<font size="6" color='red'> write API key value for client_key and API secret key value for client_secret</font>

In [1]:
client_key = 'Ej9MUEmuFZKf77fyBR1kVJixo'
client_secret = 'NBWRYAhRvCE7cjzb0RlTmqNANZEUQVDJhpeqpgQTnvxmhma466'

In [2]:
import requests
import base64

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [3]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [4]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [5]:
json_data =  response.json()
json_data

{'access_token': 'AAAAAAAAAAAAAAAAAAAAAOXR9QAAAAAAcVn9whQWl2pqGrEmtsYX9ML%2BhjI%3Dk7d6bTr3AEhrBzXX2OfEkz3CE1yVFbwKZ1AKfEqT8RlCy7oVXD',
 'token_type': 'bearer'}

In [6]:
access_token = json_data['access_token']

# Let's do some search on climate change
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [7]:
max_tweets=10000
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []

In [8]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

parameters = { 'q': 'climate change',
                    'result_type': 'recent',
                'count': 100 }

search_url = base_url+'1.1/search/tweets.json'

response = requests.get(search_url, headers=search_headers, params=parameters)

In [9]:
climate_tweets = response.json()


In [10]:
len(climate_tweets['statuses'])

100

In [11]:
tweets +=climate_tweets['statuses']


In [12]:

 ids = [tw['id'] for tw in climate_tweets['statuses']]
    

In [13]:
min_id = min(ids)

In [14]:
tw_ids = []
tw_ids+=ids

As per twitter api documentation we can use

max_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Q2 (.5 points) Modify only *parameters* dictionary in the following code so that subsequent call to climate change search returns older tweets with id less than minimum of previous tweets ids.

In [15]:
for  i in range(max_tweets//tw_limit -1):
    parameters = { 'q': 'climate change',
                    'result_type': 'recent',
                'count': tw_limit,
                  'max_id':min_id,
                 }
    print("searching tweets with id  < {}".format(min_id))
    search_url = base_url+'1.1/search/tweets.json'
    response = requests.get(search_url, headers=search_headers, params=parameters)
    climate_tweets = response.json()
    ids = [tw['id'] for tw in climate_tweets['statuses']]
    tw_ids+=ids
    print(i, len(climate_tweets['statuses']))
    min_id = min(ids)
    tweets +=climate_tweets['statuses']

searching tweets with id  < 1095388825449197569
0 100
searching tweets with id  < 1095388561149489152
1 100
searching tweets with id  < 1095388312167161858
2 100
searching tweets with id  < 1095388033233371138
3 100
searching tweets with id  < 1095387801275777030
4 100
searching tweets with id  < 1095387589446696960
5 100
searching tweets with id  < 1095387374803210241
6 100
searching tweets with id  < 1095387119801909249
7 100
searching tweets with id  < 1095386863165194240
8 100
searching tweets with id  < 1095386650061037573
9 100
searching tweets with id  < 1095386342845005824
10 100
searching tweets with id  < 1095386098593873920
11 100
searching tweets with id  < 1095385858474151939
12 100
searching tweets with id  < 1095385614223007744
13 100
searching tweets with id  < 1095385362766196736
14 100
searching tweets with id  < 1095385157207449600
15 100
searching tweets with id  < 1095384923538681857
16 100
searching tweets with id  < 1095384709285195782
17 100
searching tweets wit

In [16]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 9969


# Let's print some tweets

In [17]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [18]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Tue Feb 12 18:28:00 +0000 2019',
  'entities': {'hashtags': [],
               'symbols': [],
               'urls': [{'display_url': 'nyti.ms/2E5jpe9',
                         'expanded_url': 'https://nyti.ms/2E5jpe9',
                         'indices': [67, 90],
                         'url': 'https://t.co/DJ0A491XXN'}],
               'user_mentions': []},
  'favorite_count': 0,
  'favorited': False,
  'geo': None,
  'id': 1095389070325411842,
  'id_str': '1095389070325411842',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'is_quote_status': False,
  'lang': 'en',
  'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
  'place': None,
  'possibly_sensitive': False,
  'retweet_count': 0,
  'retweeted': False,
  'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web '
 

Link to understand [tweet-object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

Let'see some tweets

In [19]:
for status in tweets[0:max_tweet_to_print]:
    print(status['text'] + '\n')

The Tiny Swiss Company That Thinks It Can Help Stop Climate Change https://t.co/DJ0A491XXN

RT @HillaryClinton: Something extraordinary is happening in Europe—and perhaps soon in America. Teen girls are leading a movement to stop c…

RT @HillaryClinton: Something extraordinary is happening in Europe—and perhaps soon in America. Teen girls are leading a movement to stop c…

RT @HillaryClinton: Something extraordinary is happening in Europe—and perhaps soon in America. Teen girls are leading a movement to stop c…



# Pandas DataFrame for storing tweet feed

Create following date frame for storing tweets

- **climate_feed_df** DataFrame for storing tweet
- **user_df** DataFrame for storing user detail
- **retweeted_status_df** for storing original tweet

# Printing keys in tweets

In [20]:
print(tweets[1].keys())
print(len(tweets[1].keys()))


dict_keys(['in_reply_to_status_id_str', 'truncated', 'user', 'lang', 'created_at', 'entities', 'in_reply_to_user_id', 'contributors', 'in_reply_to_screen_name', 'id', 'is_quote_status', 'retweet_count', 'in_reply_to_user_id_str', 'retweeted', 'favorited', 'place', 'id_str', 'coordinates', 'retweeted_status', 'source', 'in_reply_to_status_id', 'geo', 'favorite_count', 'text', 'metadata'])
25


In [21]:
import pandas as pd

# Generate generator for building dataframe

# Q3 (2 point) Complete following   python generator.
For each tweet in tweets
- Replace **user** attribute with **user_id** and value with **tweet['user']** value.
- Replace **retweeted_status** attribute with **retweeted_status_id** and value with **tweet['retweeted_status']** value.
- Keep other attributes in the tweet as such.

In [34]:
def tweet_generator(tweets):
    for tweet in tweets:
        modified_tweet= {}
        for tw_k, tw_v in tweet.items():
            if tw_k == 'user':
                modified_tweet['user_id'] = tweet['user']['id']
            elif tw_k == 'retweeted_status':
                modified_tweet['retweeted_status_id'] = tw_v['id']
            else:
                modified_tweet[tw_k] = tw_v
            
        
        yield modified_tweet       
            
            

In [36]:
#checking tweet_generator
list(tweet_generator(tweets[0:2]))

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Tue Feb 12 18:28:00 +0000 2019',
  'entities': {'hashtags': [],
   'symbols': [],
   'urls': [{'display_url': 'nyti.ms/2E5jpe9',
     'expanded_url': 'https://nyti.ms/2E5jpe9',
     'indices': [67, 90],
     'url': 'https://t.co/DJ0A491XXN'}],
   'user_mentions': []},
  'favorite_count': 0,
  'favorited': False,
  'geo': None,
  'id': 1095389070325411842,
  'id_str': '1095389070325411842',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'is_quote_status': False,
  'lang': 'en',
  'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
  'place': None,
  'possibly_sensitive': False,
  'retweet_count': 0,
  'retweeted': False,
  'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
  'text': 'The Tiny Swiss Company That Thinks It Can Help Stop Climate Change https://

In [26]:
climate_feed_df = pd.DataFrame(tweet_generator(tweets))

In [27]:
climate_feed_df

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status_id,source,text,truncated,user_id
0,None,None,Tue Feb 12 18:28:00 +0000 2019,"{'symbols': [], 'hashtags': [], 'urls': [{'dis...",NaN,0,False,None,1095389070325411842,1095389070325411842,...,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",The Tiny Swiss Company That Thinks It Can Help...,False,28399400
1,None,None,Tue Feb 12 18:27:57 +0000 2019,"{'symbols': [], 'hashtags': [], 'urls': [], 'u...",NaN,0,False,None,1095389059101442049,1095389059101442049,...,NaN,NaN,NaN,1433,False,1.095382e+18,"<a href=""http://twitter.com/download/iphone"" r...",RT @HillaryClinton: Something extraordinary is...,False,518981025
2,None,None,Tue Feb 12 18:27:57 +0000 2019,"{'symbols': [], 'hashtags': [], 'urls': [], 'u...",NaN,0,False,None,1095389058275180544,1095389058275180544,...,NaN,NaN,NaN,1433,False,1.095382e+18,"<a href=""http://twitter.com/download/android"" ...",RT @HillaryClinton: Something extraordinary is...,False,1071189862303367168
3,None,None,Tue Feb 12 18:27:57 +0000 2019,"{'symbols': [], 'hashtags': [], 'urls': [], 'u...",NaN,0,False,None,1095389057344028672,1095389057344028672,...,NaN,NaN,NaN,1433,False,1.095382e+18,"<a href=""http://twitter.com/download/android"" ...",RT @HillaryClinton: Something extraordinary is...,False,301607325
4,None,None,Tue Feb 12 18:27:54 +0000 2019,"{'symbols': [], 'hashtags': [], 'urls': [], 'u...",NaN,0,False,None,1095389046455463938,1095389046455463938,...,NaN,NaN,NaN,45,False,1.095255e+18,"<a href=""http://twitter.com/download/iphone"" r...","RT @RobGMacfarlane: “Of all risks, it's in rel...",False,702239479
5,None,None,Tue Feb 12 18:27:53 +0000 2019,"{'symbols': [], 'hashtags': [], 'urls': [{'dis...",NaN,0,False,None,1095389041934168064,1095389041934168064,...,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",UK Government: Declare a national climate emer...,False,1256900354
6,None,None,Tue Feb 12 18:27:53 +0000 2019,"{'symbols': [], 'hashtags': [], 'urls': [], 'u...",NaN,0,False,None,1095389040499539970,1095389040499539970,...,NaN,NaN,NaN,8,False,1.095343e+18,"<a href=""http://twitter.com/download/android"" ...",RT @RepSpeier: America is at a crossroads: We ...,False,418348168
7,None,None,Tue Feb 12 18:27:52 +0000 2019,"{'symbols': [], 'hashtags': [], 'urls': [], 'u...",NaN,0,False,None,1095389039186849792,1095389039186849792,...,NaN,NaN,NaN,47,False,1.095375e+18,"<a href=""http://twitter.com/download/iphone"" r...",RT @kenklippenstein: As House Dems plan for a ...,False,135610784
8,None,None,Tue Feb 12 18:27:52 +0000 2019,"{'symbols': [], 'hashtags': [], 'urls': [], 'u...",NaN,0,False,None,1095389037802807296,1095389037802807296,...,NaN,NaN,NaN,1433,False,1.095382e+18,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @HillaryClinton: Something extraordinary is...,False,59164080
9,None,None,Tue Feb 12 18:27:52 +0000 2019,"{'symbols': [], 'hashtags': [], 'urls': [], 'u...",NaN,0,False,None,1095389036288471041,1095389036288471041,...,NaN,NaN,NaN,203,False,1.095357e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @RLong_Bailey: In 2019 Labour will run an u...,False,713833849705201664


# Q4(1 points) List datatype using climate_feed_df. List which attributes doesn't have right data types.

In [68]:
climate_feed_df.dtypes

contributors                  object
coordinates                   object
created_at                    object
entities                      object
favorite_count                object
favorited                     object
geo                           object
id                            object
id_str                        object
in_reply_to_screen_name       object
in_reply_to_status_id         object
in_reply_to_status_id_str     object
in_reply_to_user_id           object
in_reply_to_user_id_str       object
is_quote_status               object
lang                          object
metadata                      object
place                         object
possibly_sensitive            object
quoted_status_id              object
quoted_status_id_str          object
retweet_count                 object
retweeted                     object
retweeted_status             float64
source                        object
text                          object
truncated                     object
u

# write columns with wrong datatype and mention correct data type too.

Make sure integer , float, data time etc. is correctly converted

# Q5 (1 point) Complete following tweet user generator.

In [216]:
def tweet_user_generator(tweets):
    for status in tweets:
        modified_tweet = {}
        for tweet_attr, tweet_val in  status.items():            
            if tweet_attr == "user":
                modified_tweet = tweet_val
        yield modified_tweet      

In [217]:
user_df = pd.DataFrame(tweet_user_generator(tweets))

In [218]:
user_df

,contributors_enabled,created_at,default_profile,default_profile_image,description,entities,favourites_count,follow_request_sent,followers_count,following,friends_count,geo_enabled,has_extended_profile,id,id_str,is_translation_enabled,is_translator,lang,listed_count,location,name,notifications,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_banner_url,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,time_zone,translator_type,url,utc_offset,verified
0,False,Wed Apr 15 18:26:27 +0000 2009,False,False,Environ. policy & #humanrights advocate (MN/WI...,"{'description': {'urls': []}, 'url': {'urls': ...",24941,None,1042,None,1939,True,False,31476142,31476142,False,False,en,70,"Caledonia, MN / Tri-State Area",Janette Dean,None,ACDED6,http://abs.twimg.com/images/themes/theme18/bg.gif,https://abs.twimg.com/images/themes/theme18/bg...,True,https://pbs.twimg.com/profile_banners/31476142...,http://pbs.twimg.com/profile_images/7673989647...,https://pbs.twimg.com/profile_images/767398964...,1B95E0,000000,000000,000000,True,False,JanetteD,18763,None,none,https://t.co/vWN0L6b7K0,None,False
1,False,Sat Dec 08 09:17:28 +0000 2012,False,False,"Citizen of the world. Dad, husband, ex husband...","{'description': {'urls': []}, 'url': {'urls': ...",14258,None,734,None,1542,True,False,996782298,996782298,False,False,en,44,Somewhere on the North Coast,Ross Radburn,None,000000,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/99678229...,http://pbs.twimg.com/profile_images/8850683235...,https://pbs.twimg.com/profile_images/885068323...,1B95E0,000000,000000,000000,False,False,Windbornwarrior,46038,None,none,https://t.co/ERrC3CmAjz,None,False
2,False,Sun Jun 07 12:50:45 +0000 2009,False,False,,{'description': {'urls': []}},179413,None,3968,None,4346,False,False,45333453,45333453,False,False,en,3,Alabama,❌Thomas ⭐⭐⭐💯🇺🇸,None,000000,http://abs.twimg.com/images/themes/theme6/bg.gif,https://abs.twimg.com/images/themes/theme6/bg.gif,False,NaN,http://pbs.twimg.com/profile_images/274553105/...,https://pbs.twimg.com/profile_images/274553105...,3B94D9,000000,000000,000000,False,False,bass460,52663,None,none,None,None,False
3,False,Thu Jul 19 22:07:52 +0000 2018,False,False,Bored at work. Needed something to do. Post tw...,{'description': {'urls': []}},10129,None,379,None,936,False,True,1020067726532829185,1020067726532829185,False,False,en,3,"Texas, USA",Cuban Bot,None,000000,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,NaN,http://pbs.twimg.com/profile_images/1033497344...,https://pbs.twimg.com/profile_images/103349734...,19CF86,000000,000000,000000,False,False,RC91681842,13480,None,none,None,None,False
4,False,Mon Feb 02 02:28:11 +0000 2009,False,False,here for my friends. bayern munich&german nt. ...,"{'description': {'urls': []}, 'url': {'urls': ...",515015,None,682,None,506,False,False,19889325,19889325,False,False,en,10,"FL, USA",Dee,None,2C0647,http://abs.twimg.com/images/themes/theme10/bg.gif,https://abs.twimg.com/images/themes/theme10/bg...,True,https://pbs.twimg.com/profile_banners/19889325...,http://pbs.twimg.com/profile_images/1079438300...,https://pbs.twimg.com/profile_images/107943830...,ED0909,FFFFFF,7AC3EE,3D1957,True,False,weronlystardust,168060,None,none,https://t.co/sq4uWkbweL,None,False
5,False,Wed Oct 22 17:37:56 +0000 2008,True,False,"Writer, editor, educator; Cal Berkeley alum #U...",{'description': {'urls': []}},415124,None,5427,None,5793,False,False,16909343,16909343,False,False,en,7,"Portland, OR",bethlevin,None,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/16909343

# Q 6(1 point) List datatype using user_df. List which attributes doesn't have right data types.

In [85]:
user_df.dtypes

contributors_enabled                  object
created_at                            object
default_profile                       object
default_profile_image                 object
description                           object
entities                              object
favourites_count                      object
follow_request_sent                   object
followers_count                       object
following                             object
friends_count                         object
geo_enabled                           object
has_extended_profile                  object
id                                    object
id_str                                object
is_translation_enabled                object
is_translator                         object
lang                                  object
listed_count                          object
location                              object
name                                  object
notifications                         object
profile_ba

Similar to previous question

# Q7(1 point) Complete following retweeted_status_generator .Generator should yield dictionary of retweeted_status value from each tweet. If a tweet doesn't have this key return empty dictionary

In [219]:
def retweeted_status_generator(tweets):
    for status in tweets:
        modified_tweet = {}
        for tweet_attr, tweet_val in  status.items():            
            if tweet_attr == "retweeted_status":
                modified_tweet = tweet_val
        yield modified_tweet  

In [221]:
retweeted_status_df = pd.DataFrame(retweeted_status_generator(tweets))

In [222]:
retweeted_status_df

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,metadata,place,possibly_sensitive,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,scopes,source,text,truncated,user
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,None,Sat Feb 09 20:27:45 +0000 2019,"{'hashtags': [], 'user_mentions': [], 'urls': ...",NaN,99.0,False,None,1.094332e+18,1094332044052549633,None,NaN,None,NaN,None,False,en,"{'result_type': 'recent', 'iso_language_code':...",None,False,NaN,NaN,NaN,81.0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",The National Party declares war on the Glouces...,True,"{'default_profile': False, 'protected': False,..."
2,NaN,None,Sat Feb 09 23:56:18 +0000 2019,"{'hashtags': [], 'user_mentions': [], 'urls': ...",NaN,6.0,False,None,1.094385e+18,1094384528267902977,None,NaN,None,NaN,None,True,en,"{'result_type': 'recent', 'iso_language_code':...",None,False,"{'favorited': False, 'favorite_count': 2642, '...",1.093999e+18,1093998640392220673,5.0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",This is the same idiot who thought\nThe southe...,False,"{'default_profile': False, 'protected': False,..."
3,NaN,None,Sat Feb 09 23:34:27 +0000 2019,"{'hashtags': [], 'user_mentions': [], 'urls': ...",NaN,190.0,False,None,1.094379e+18,1094379028608438272,None,NaN,None,NaN,None,True,en,"{'result_type': 'recent', 'iso_language_code':...",None,False,"{'favorited': False, 'favorite_count': 220, 'i...",1.093932e+18,1093931686998487040,31.0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",Communism is Fascism. Fighting Climate Change ...,True,"{'default_profile': False, 'protected': False,..."
4,NaN,None,Sat Feb 09 19:07:56 +0000 2019,"{'hashtags': [], 'user_mentions': [], 'urls': ...",NaN,2905.0,False,None,1.094312e+18,1094311957669265408,None,NaN,None,NaN,None,False,en,"{'result_type': 'recent', 'iso_language_code':...",None,NaN,NaN,NaN,NaN,599.0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Unmitigated climate change would literally des...,True,"{'default_profile': True, 'protected': False, ..."
5,NaN,None,Sat Feb 09 23:41:20 +0000 2019,"{'hashtags': [], 'user_mentions': [{'id_str': ...",NaN,100.0,False,None,1.094381e+18,1094380759719260162,realDonaldTrump,1.094376e+18,1094375749279248385,2.507388e+07,25073877,False,en,"{'result_type': 'recent', 'iso_language_code':...",None,NaN,NaN,NaN,NaN,14.0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@realDonaldTrump Climate change denial has not...,True,"{'default_profile': False, 'protected': False,..."
6,NaN,None,Sat Feb 09 16:44:41 +0000 2019,"{'hashtags': [], 'user_mentions': [], 'urls': ...",NaN,2290.0,False,None,1.094276e+18,1094275905378308096,None,NaN,None,NaN,None,False,en,"{'result_type': 'recent', 'iso_language_code':...",None,NaN,NaN,NaN,NaN,447.0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Happy Saturday! Friendly reminder that climate...,False,"{'default_profile': True, 'protected': False, ..."
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,None,Sat Feb 09 20:41:00 +0000 2019,"{'hashtags': [], 'user_mentions': [], 'urls': ...",NaN,166.0,False,None,1.094335e+18,1094335377194799104,None,NaN,None,NaN,None,False,en,"{'result_type': 'recent', 'iso_language_code':...",None,False,NaN,NaN,NaN,79.0,False,NaN,"<a href=""https://buffer.com"" rel=""nofollow"">Bu...",Blue marble no more? https://t.co/hFLndZsiBB,False,"{'default_profile': False, 'protected': False,..."
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Q8(1 point) Write code to find number of rows in retweeted_status_df with all the values Nan

In [7]:
import numpy as np
import pandas as pd

In [225]:
sum(np.all(retweeted_status_df.isna(), axis=1))

2690

In [14]:
na_status=retweeted_status_df.isna()

In [17]:
na_status

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,scopes,source,text,truncated,user
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,False,False,True,False,False,True,False,False,...,True,True,True,False,False,True,False,False,False,False
2,True,True,False,False,True,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
3,True,True,False,False,True,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
4,True,True,False,False,True,False,False,True,False,False,...,True,True,True,False,False,True,False,False,False,False
5,True,True,False,False,True,False,False,True,False,False,...,True,True,True,False,False,True,False,False,False,False
6,True,True,False,False,True,False,False,True,False,False,...,True,True,True,False,False,True,False,False,False,False
7,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
8,True,True,False,False,True,False,False,True,False,False,...,True,True,True,False,False,True,False,False,False,False
9,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [21]:
na_status.all(axis= 'columns').sum()

2690

# ignore this part

In [226]:
#retweeted_status_df.to_pickle("./retweeted_status_df.pkl")

In [11]:
retweeted_status_df = pd.read_pickle("./twitter_data/retweeted_status_df.pkl")

In [12]:
retweeted_status_df

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,scopes,source,text,truncated,user
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,None,Sat Feb 09 20:27:45 +0000 2019,"{'symbols': [], 'urls': [{'display_url': 'twit...",NaN,99.0,False,None,1.094332e+18,1094332044052549633,...,NaN,NaN,NaN,81.0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",The National Party declares war on the Glouces...,True,{'created_at': 'Wed Oct 15 07:27:24 +0000 2008...
2,NaN,None,Sat Feb 09 23:56:18 +0000 2019,"{'symbols': [], 'urls': [{'display_url': 'twit...",NaN,6.0,False,None,1.094385e+18,1094384528267902977,...,"{'in_reply_to_user_id_str': None, 'in_reply_to...",1.093999e+18,1093998640392220673,5.0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",This is the same idiot who thought\nThe southe...,False,{'created_at': 'Thu Apr 06 23:39:57 +0000 2017...
3,NaN,None,Sat Feb 09 23:34:27 +0000 2019,"{'symbols': [], 'urls': [{'display_url': 'twit...",NaN,190.0,False,None,1.094379e+18,1094379028608438272,...,"{'in_reply_to_user_id_str': None, 'in_reply_to...",1.093932e+18,1093931686998487040,31.0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",Communism is Fascism. Fighting Climate Change ...,True,{'created_at': 'Tue Jul 12 23:19:48 +0000 2016...
4,NaN,None,Sat Feb 09 19:07:56 +0000 2019,"{'symbols': [], 'urls': [{'display_url': 'twit...",NaN,2905.0,False,None,1.094312e+18,1094311957669265408,...,NaN,NaN,NaN,599.0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Unmitigated climate change would literally des...,True,{'created_at': 'Wed Jul 31 01:41:01 +0000 2013...
5,NaN,None,Sat Feb 09 23:41:20 +0000 2019,"{'symbols': [], 'urls': [{'display_url': 'twit...",NaN,100.0,False,None,1.094381e+18,1094380759719260162,...,NaN,NaN,NaN,14.0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@realDonaldTrump Climate change denial has not...,True,{'created_at': 'Wed Jun 22 03:23:57 +0000 2011...
6,NaN,None,Sat Feb 09 16:44:41 +0000 2019,"{'symbols': [], 'urls': [], 'hashtags': [], 'u...",NaN,2290.0,False,None,1.094276e+18,1094275905378308096,...,NaN,NaN,NaN,447.0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Happy Saturday! Friendly reminder that climate...,False,{'created_at': 'Tue Feb 21 20:46:09 +0000 2012...
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,None,Sat Feb 09 20:41:00 +0000 2019,"{'symbols': [], 'urls': [{'display_url': 'po.s...",NaN,166.0,False,None,1.094335e+18,1094335377194799104,...,NaN,NaN,NaN,79.0,False,NaN,"<a href=""https://buffer.com"" rel=""nofollow"">Bu...",Blue marble no more? https://t.co/hFLndZsiBB,False,{'created_at': 'Thu Jan 13 19:33:56 +0000 2011...
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [230]:
#user_df.to_pickle("./user.pkl")
unpickled_user_df = pd.read_pickle("./user.pkl")
unpickled_user_df

,contributors_enabled,created_at,default_profile,default_profile_image,description,entities,favourites_count,follow_request_sent,followers_count,following,friends_count,geo_enabled,has_extended_profile,id,id_str,is_translation_enabled,is_translator,lang,listed_count,location,name,notifications,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_banner_url,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,time_zone,translator_type,url,utc_offset,verified
0,False,Wed Apr 15 18:26:27 +0000 2009,False,False,Environ. policy & #humanrights advocate (MN/WI...,"{'description': {'urls': []}, 'url': {'urls': ...",24941,None,1042,None,1939,True,False,31476142,31476142,False,False,en,70,"Caledonia, MN / Tri-State Area",Janette Dean,None,ACDED6,http://abs.twimg.com/images/themes/theme18/bg.gif,https://abs.twimg.com/images/themes/theme18/bg...,True,https://pbs.twimg.com/profile_banners/31476142...,http://pbs.twimg.com/profile_images/7673989647...,https://pbs.twimg.com/profile_images/767398964...,1B95E0,000000,000000,000000,True,False,JanetteD,18763,None,none,https://t.co/vWN0L6b7K0,None,False
1,False,Sat Dec 08 09:17:28 +0000 2012,False,False,"Citizen of the world. Dad, husband, ex husband...","{'description': {'urls': []}, 'url': {'urls': ...",14258,None,734,None,1542,True,False,996782298,996782298,False,False,en,44,Somewhere on the North Coast,Ross Radburn,None,000000,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/99678229...,http://pbs.twimg.com/profile_images/8850683235...,https://pbs.twimg.com/profile_images/885068323...,1B95E0,000000,000000,000000,False,False,Windbornwarrior,46038,None,none,https://t.co/ERrC3CmAjz,None,False
2,False,Sun Jun 07 12:50:45 +0000 2009,False,False,,{'description': {'urls': []}},179413,None,3968,None,4346,False,False,45333453,45333453,False,False,en,3,Alabama,❌Thomas ⭐⭐⭐💯🇺🇸,None,000000,http://abs.twimg.com/images/themes/theme6/bg.gif,https://abs.twimg.com/images/themes/theme6/bg.gif,False,NaN,http://pbs.twimg.com/profile_images/274553105/...,https://pbs.twimg.com/profile_images/274553105...,3B94D9,000000,000000,000000,False,False,bass460,52663,None,none,None,None,False
3,False,Thu Jul 19 22:07:52 +0000 2018,False,False,Bored at work. Needed something to do. Post tw...,{'description': {'urls': []}},10129,None,379,None,936,False,True,1020067726532829185,1020067726532829185,False,False,en,3,"Texas, USA",Cuban Bot,None,000000,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,NaN,http://pbs.twimg.com/profile_images/1033497344...,https://pbs.twimg.com/profile_images/103349734...,19CF86,000000,000000,000000,False,False,RC91681842,13480,None,none,None,None,False
4,False,Mon Feb 02 02:28:11 +0000 2009,False,False,here for my friends. bayern munich&german nt. ...,"{'description': {'urls': []}, 'url': {'urls': ...",515015,None,682,None,506,False,False,19889325,19889325,False,False,en,10,"FL, USA",Dee,None,2C0647,http://abs.twimg.com/images/themes/theme10/bg.gif,https://abs.twimg.com/images/themes/theme10/bg...,True,https://pbs.twimg.com/profile_banners/19889325...,http://pbs.twimg.com/profile_images/1079438300...,https://pbs.twimg.com/profile_images/107943830...,ED0909,FFFFFF,7AC3EE,3D1957,True,False,weronlystardust,168060,None,none,https://t.co/sq4uWkbweL,None,False
5,False,Wed Oct 22 17:37:56 +0000 2008,True,False,"Writer, editor, educator; Cal Berkeley alum #U...",{'description': {'urls': []}},415124,None,5427,None,5793,False,False,16909343,16909343,False,False,en,7,"Portland, OR",bethlevin,None,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/16909343

In [231]:
#climate_feed_df.to_pickle("./climate_feed.pkl")
unpickled_climate_feed_df = pd.read_pickle("./climate_feed.pkl")
unpickled_climate_feed_df

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,metadata,place,possibly_sensitive,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,truncated,user_id
0,None,None,Sun Feb 10 00:17:31 +0000 2019,"{'hashtags': [], 'user_mentions': [{'id_str': ...",NaN,0,False,None,1094389867499405313,1094389867499405313,None,None,None,None,None,False,en,"{'result_type': 'recent', 'iso_language_code':...",None,False,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",.@SteveKingIA says he's unsure about the scien...,True,31476142
1,None,None,Sun Feb 10 00:17:30 +0000 2019,"{'hashtags': [], 'user_mentions': [{'id_str': ...",NaN,0,False,None,1094389862193455105,1094389862193455105,None,None,None,None,None,False,en,"{'result_type': 'recent', 'iso_language_code':...",None,NaN,NaN,NaN,NaN,81,False,1.094332e+18,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",RT @RobOakeshott1: The National Party declares...,False,996782298
2,None,None,Sun Feb 10 00:17:26 +0000 2019,"{'hashtags': [], 'user_mentions': [{'id_str': ...",NaN,0,False,None,1094389844078485506,1094389844078485506,None,None,None,None,None,True,en,"{'result_type': 'recent', 'iso_language_code':...",None,False,NaN,1093998640392220673,1093998640392220673,5,False,1.094385e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @emilia_suze: This is the same idiot who th...,False,45333453
3,None,None,Sun Feb 10 00:17:24 +0000 2019,"{'hashtags': [], 'user_mentions': [{'id_str': ...",NaN,0,False,None,1094389837522599936,1094389837522599936,None,None,None,None,None,True,en,"{'result_type': 'recent', 'iso_language_code':...",None,NaN,NaN,1093931686998487040,1093931686998487040,31,False,1.094379e+18,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @PrettyBadLefty: Communism is Fascism. Figh...,False,1020067726532829185
4,None,None,Sun Feb 10 00:17:23 +0000 2019,"{'hashtags': [], 'user_mentions': [{'id_str': ...",NaN,0,False,None,1094389831419994112,1094389831419994112,None,None,None,None,None,False,en,"{'result_type': 'recent', 'iso_language_code':...",None,NaN,NaN,NaN,NaN,599,False,1.094312e+18,"<a href=""http://twitter.com/download/android"" ...",RT @kenklippenstein: Unmitigated climate chang...,False,19889325
5,None,None,Sun Feb 10 00:17:21 +0000 2019,"{'hashtags': [], 'user_mentions': [{'id_str': ...",NaN,0,False,None,1094389825266900998,1094389825266900998,None,None,None,None,None,False,en,"{'result_type': 'recent', 'iso_language_code':...",None,NaN,NaN,NaN,NaN,14,False,1.094381e+18,"<a href=""http://twitter.com/download/iphone"" r...",RT @ChrisJZullo: @realDonaldTrump Climate chan...,False,16909343
6,None,None,Sun Feb 10 00:17:21 +0000 2019,"{'hashtags': [], 'user_mentions': [{'id_str': ...",NaN,0,False,None,1094389825132810245,1094389825132810245,None,None,None,None,None,False,en,"{'result_type': 'recent', 'iso_language_code':...",None,NaN,NaN,NaN,NaN,447,False,1.094276e+18,"<a href=""http://twitter.com/download/android"" ...",RT @alexanderbruz: Happy Saturday! Friendly re...,False,908017796881240064
7,None,None,Sun Feb 10 00:17:20 +0000 2019,"{'hashtags': [], 'user_mentions': [{'id_str': ...",NaN,0,False,None,1094389819239686144,1094389819239686144,kantarjiev,1094387771387535360,1094387771387535360,45702582,45702582,False,en,"{'result_type': 'recent', 'iso_language_code':...",None,False,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...","@kantarjiev @gsheyner Yes, this downtown parki...",True,1065520951150800896
8,None,None,Sun Feb 10 00:17:19 +0000 2019,"{'hashtags': [], 'user_mentions': [{'id_str': ...",NaN,0,False,None,1094389815141982211,1094389815141982211,None,None,None,None,None,False,en,"{'result_type': 'recent', 'iso_language_code':...",None,True,NaN,NaN,NaN,79,False,1.